# Functional Preprocessing

This notebooks preprocesses functional MRI images. It requires that the anatomical preprocessing workflwo was already executed. This functional preprocessing workflow runs the following processing steps:

1. Reorient Images to RAS
1. Removal of non-steady state volumes 
1. Motion Correction with SPM
1. Slice-wise Correction with SPM
1. Brain Extraction with SPM and FSL
1. High-Pass Filter with FSL
1. Two- step coregistration using BBR with FSL, using WM segmentation from SPM
1. Low-pass and band-pass smoothing with Nilearn

Additional, this workflow also performs:
 - ACompCor
 - TCompCor
 - Artifact Detection
 - Computes Friston's 24-paramter model for motion parameters

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format. Note that the data should be in a session subfolder:

    dataset
    ├── analysis-func_task-{task_id}_specs.json
    ├── sub-{sub_id}
    │   └── ses-{sess_id}
    │       └── func
    │           ├── sub-{sub_id}_ses-{sess_id}_task-{task_id}_run-{run_id}_bold.nii.gz
    └── task-{task_id}_bold.json

## Execution Specifications

This notebook will extract the relevant processing specifications from the `analysis-func_task-{task_id}_specs.json` file in the dataset folder. In the current setup, they are as follows:

In [ ]:
# Name of task to process
task_id = 'fingerfootlips'

import json
from os.path import join as opj

spec_file = opj('/data', 'analysis-func_task-%s_specs.json' % task_id)

with open(spec_file) as f:
    specs = json.load(f)

specs

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject names
subject_list = specs['subject_list']

# List of session names
session_list = specs['session_list']

# List of run names
run_list = specs['run_list']

# Width of smoothing kernels to apply
fwhm = specs['fwhm']

# Width of band-pass smoothing kernel to apply
fwhm_bp = specs['fwhm_bp']

# Requested isometric voxel resolution after coregistration
voxel_res = specs['voxel_res']

# Reference Slice or time point (in ms) required for slice wise correction
ref_slice = specs['ref_slice']

# High pass filter [in seconds] to apply
hpf = specs['hpf']

# Number of components to extract with ACompCor and TCompCor
ncomp = specs['ncomp']

# Number of cores to use
n_proc = specs['n_proc']

# Cerating the Workflow

To ensure a good overview of the functional preprocessing, the workflow was divided into three subworkflows:

1. The Main Workflow, i.e. doing the actual preprocessing
2. The Confound Workflow, i.e. computing confound variables
3. Visualization Workflow, i.e. visualizating relevant steps for quality control

## Import Modules

In [ ]:
import os
import numpy as np
from os.path import join as opj
from nipype import Workflow, Node, IdentityInterface, Function
from nipype.interfaces.image import Reorient
from nipype.interfaces.spm import SliceTiming, Realign, Smooth
from nipype.interfaces.fsl import FLIRT, MeanImage, BET, TemporalFilter, BinaryMaths, ExtractROI
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.algorithms.confounds import ACompCor, TCompCor, NonSteadyStateDetector

# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-dev/spm12_mcr/spm/spm12')

## Relevant Execution Variables

In [ ]:
# Relevant folder paths and names
exp_dir = '/output'
out_dir = 'datasink'
work_dir = 'workingdir'

## Create a subworkflow for the Main Workflow

### Implement Nodes

In [ ]:
# Reorient anatomical images to RAS
reorient = Node(Reorient(orientation='RAS'), name='reorient')

In [ ]:
# Detection of Non-Steady State volumes
nonsteady_detection = Node(NonSteadyStateDetector(), name='nonsteady_detection')

In [ ]:
# Removal of Non-Steady State volumes
nonsteady_removal = Node(ExtractROI(output_type='NIFTI',
                                    t_size=-1),
                         name='nonsteady_removal')

In [ ]:
# Extract sequence specifications of functional images
def get_parameters(task_id):
    
    import json
    import numpy as np
    from os.path import join as opj
        
    func_desc = opj('/data', 'task-%s_bold.json' % task_id)

    with open(func_desc) as f:
        func_desc = json.load(f)

    # Read out relevant parameters
    TR = func_desc['RepetitionTime']
    slice_order = func_desc['SliceTiming']
    nslices = len(slice_order)
    time_acquisition = float(TR)-(TR/nslices)
    
    return TR, slice_order, nslices, time_acquisition

getParam = Node(Function(input_names=['task_id'],
                         output_names=['TR', 'slice_order',
                                       'nslices', 'time_acquisition'],
                         function=get_parameters),
                name='getParam')

In [ ]:
# Correct for motion
realign = Node(Realign(register_to_mean=True), name='realign')

In [ ]:
# Correct for slice-wise acquisition
slicetime = Node(SliceTiming(ref_slice=ref_slice), name='slicetime')

In [ ]:
# Reset TR value after SPM's slice time correction
def add_TR_to_file(in_file, TR):
    
    import nibabel as nb
    
    # Load image
    img = nb.load(in_file)
    
    # Reset TR
    img.header.set_zooms(list(img.header.get_zooms()[:3]) + [TR])
    
    # Save file
    out_file = in_file.replace('.nii', '_TR.nii')
    img.to_filename(out_file)
    del img
    
    return out_file

reset_TR = Node(Function(input_names=['in_file', 'TR'],
                         output_names=['out_file'],
                         function=add_TR_to_file),
                name='reset_TR')

In [ ]:
# Remove skull signal from functional images
bet_func = Node(BET(functional=True,
                    mask=True,
                    output_type='NIFTI_GZ'),
                name='bet_func')

In [ ]:
# Apply high-pass filter to data
hp_filter = Node(TemporalFilter(output_type='NIFTI_GZ'),
                 name='hp_filter')

In [ ]:
# Compute sigma for temporal high-pass filter
def get_sigma(TR, hpf):
    return float(hpf)/float(TR)/2.

hpf_sigma = Node(Function(input_names=['TR', 'hpf'],
                          output_names=['sigma'],
                          function=get_sigma),
                 name='hpf_sigma')
hpf_sigma.inputs.hpf = hpf

In [ ]:
# Compute mean image before high-pass filter
hp_mean = Node(MeanImage(dimension='T',
                         output_type='NIFTI_GZ'),
                  name='hp_mean')

In [ ]:
# Add  mean to high-passed filtered signal
hp_combine = Node(BinaryMaths(operation='add',
                              output_type='NIFTI_GZ'),
                  name='hp_combine')

In [ ]:
# Pre-alignment of functional images to anatomical image
coreg_pre = Node(FLIRT(dof=6,
                       output_type='NIFTI_GZ'),
                 name='coreg_pre')

In [ ]:
# Coregistration of functional images to anatomical image with BBR
# using WM segmentation
coreg_bbr = Node(FLIRT(dof=6,
                       cost='bbr',
                       schedule=opj(os.getenv('FSLDIR'),
                                    'etc/flirtsch/bbr.sch'),
                       output_type='NIFTI_GZ'),
                 name='coreg_bbr')

In [ ]:
# Apply coregistration warp to functional images
applycoreg = Node(FLIRT(interp='spline',
                        apply_isoxfm=voxel_res,
                        datatype='short',
                        output_type='NIFTI_GZ'),
                 name='applycoreg')

In [ ]:
# Crop coregistered files to reduce file size
def crop_img(in_file):
    
    from nilearn.image import crop_img

    # Crop image
    out_file = in_file.replace('.nii', '_crop.nii')
    crop_img(in_file).to_filename(out_file)
    
    return out_file

cropper = Node(Function(input_names=['in_file'],
                        output_names=['out_file'],
                        function=crop_img),
               name='cropper')

In [ ]:
# Applies gaussian spatial filter as in Sengupta, Pollmann & Hanke, 2018
def gaussian_spatial_filter(in_file, fwhm, ftype='LP', bandwidth=1):

    from nilearn.image import smooth_img
    from nibabel.nifti1 import Nifti1Image

    if ftype == 'LP':
        img = smooth_img(in_file, fwhm=fwhm)
        
    elif ftype == 'BP':
        LPF_bold_1 = smooth_img(in_file, fwhm=fwhm)
        LPF_bold_2 = smooth_img(in_file, fwhm=fwhm - bandwidth)
        BPF_bold = LPF_bold_2.get_fdata() - LPF_bold_1.get_fdata()
        img = Nifti1Image(BPF_bold, LPF_bold_1.affine, LPF_bold_1.header)

    # Save and return output file
    out_file = in_file.replace('.nii', '_%s_%smm.nii' % (ftype, fwhm))
    img.to_filename(out_file)
    del img

    return out_file

# Spatial Band-Pass Filter
smooth_lowpass = Node(Function(input_names=['in_file', 'fwhm', 'ftype'],
                                  output_names=['out_file'],
                                  function=gaussian_spatial_filter),
                         name='smooth_lowpass')
smooth_lowpass.inputs.ftype = 'LP'
smooth_lowpass.iterables = ('fwhm', fwhm)

# Spatial Band-Pass Filter
smooth_bandpass = Node(Function(input_names=['in_file', 'fwhm', 'ftype'],
                                   output_names=['out_file'],
                                   function=gaussian_spatial_filter),
                          name='smooth_bandpass')
smooth_bandpass.inputs.ftype = 'BP'
smooth_bandpass.inputs.fwhm = fwhm_bp

In [ ]:
# Computes mean image
meanimg = Node(MeanImage(dimension='T',
                         output_type='NIFTI_GZ'),
               name='meanimg')

meanimg_bp = meanimg.clone('meanimg_bp')

### Create Main Workflow

**Note:** Slice time correction is applied after motion correction, as recommended by Power et al. (2017): http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0182939

In [ ]:
# Create main preprocessing workflow
mainflow = Workflow(name='mainflow')

In [ ]:
# Add nodes to workflow and connect them
mainflow.connect([(reorient, nonsteady_detection, [('out_file', 'in_file')]),
                  (reorient, nonsteady_removal, [('out_file', 'in_file')]),
                  (nonsteady_detection, nonsteady_removal, [('n_volumes_to_discard',
                                                             't_min')]),
                  (nonsteady_removal, realign, [('roi_file', 'in_files')]),
                  (realign, slicetime, [('realigned_files', 'in_files')]),
                  (getParam, slicetime, [('TR', 'time_repetition'),
                                         ('slice_order', 'slice_order'),
                                         ('nslices', 'num_slices'),
                                         ('time_acquisition', 'time_acquisition'),
                                         ]),
                  (slicetime, reset_TR, [('timecorrected_files', 'in_file')]),
                  (getParam, reset_TR, [('TR', 'TR')]),
                  (reset_TR, bet_func, [('out_file', 'in_file')]),

                  # Highpass filter
                  (getParam, hpf_sigma, [('TR', 'TR')]),
                  (hpf_sigma, hp_filter, [('sigma', 'highpass_sigma')]),
                  (bet_func, hp_filter, [('out_file', 'in_file')]),
                  (bet_func, hp_mean, [('out_file', 'in_file')]),
                  (hp_filter, hp_combine, [('out_file', 'in_file')]),
                  (hp_mean, hp_combine, [('out_file', 'operand_file')]),

                  # Coregistration
                  (coreg_pre, coreg_bbr, [('out_matrix_file', 'in_matrix_file')]),
                  (coreg_bbr, applycoreg, [('out_matrix_file', 'in_matrix_file')]),
                  (hp_mean, coreg_pre, [('out_file', 'in_file')]),
                  (hp_mean, coreg_bbr, [('out_file', 'in_file')]),
                  (hp_combine, applycoreg, [('out_file', 'in_file')]),
                  (applycoreg, cropper, [('out_file', 'in_file')]),
                  
                  # Smoothing
                  (cropper, smooth_lowpass, [('out_file', 'in_file')]),
                  (cropper, smooth_bandpass, [('out_file', 'in_file')]),

                  # Create mean image
                  (smooth_lowpass, meanimg, [('out_file', 'in_file')]),
                  (smooth_bandpass, meanimg_bp, [('out_file', 'in_file')]),
                  ])

## Create a subworkflow for the Confound Workflow

### Implement Nodes

In [ ]:
# Run ACompCor and TCompcor (based on Behzadi et al., 2007)
aCompCor = Node(ACompCor(num_components=ncomp,
                         pre_filter=False,
                         save_pre_filter=False,
                         merge_method='union',
                         components_file='compcorA.txt'),
                name='aCompCor')

tCompCor = Node(TCompCor(num_components=ncomp,
                         percentile_threshold=0.02,
                         pre_filter=False,
                         save_pre_filter=False,
                         components_file='compcorT.txt'),
                name='tCompCor')

In [ ]:
# Create binary mask for ACompCor (based on Behzadi et al., 2007)
def get_csf_wm_mask(wm, csf, in_file):
    
    from nibabel import Nifti1Image
    from nilearn.image import threshold_img, resample_to_img
    from scipy.ndimage.morphology import binary_erosion, binary_closing

    # Create eroded WM binary mask
    thr_wm = threshold_img(wm, 0.99)
    res_wm = resample_to_img(thr_wm, in_file)
    bin_wm = threshold_img(res_wm, 0.5)
    mask_wm = binary_erosion(bin_wm.get_fdata(), iterations=2).astype('int8')

    # Create eroded CSF binary mask (differs from Behzadi et al., 2007)
    thr_csf = threshold_img(csf, 0.99)
    res_csf = resample_to_img(thr_csf, in_file)
    bin_csf = threshold_img(res_csf, 0.5)
    close_csf = binary_closing(bin_csf.get_fdata(), iterations=1)
    mask_csf = binary_erosion(close_csf, iterations=1).astype('int8')
    
    # Combine WM and CSF binary masks into one
    binary_mask = ((mask_wm + mask_csf) > 0).astype('int8')
    out_file = in_file.replace('.nii', '_maskA.nii')
    Nifti1Image(binary_mask, res_wm.affine).to_filename(out_file)

    return out_file

acomp_masks = Node(Function(input_names=['wm', 'csf', 'in_file'],
                            output_names=['out_file'],
                            function=get_csf_wm_mask),
                   name='acomp_masks')

In [ ]:
# Create binary mask for TCompCor approach (based on Behzadi et al., 2007)
def get_brainmask(in_file):
    
    from nibabel import Nifti1Image
    from nilearn.image import mean_img
    from scipy.ndimage.morphology import binary_erosion
    
    img = mean_img(in_file)
    erod_img = binary_erosion(img.get_fdata()>0, iterations=1).astype('int8')
    
    out_file = in_file.replace('.nii', '_maskT.nii')
    Nifti1Image(erod_img, img.affine).to_filename(out_file)
    
    return out_file

tcomp_brainmask = Node(Function(input_names=['in_file'],
                          output_names=['out_file'],
                          function=get_brainmask),
                 name='tcomp_brainmask')

In [ ]:
# Create Confound Correlation Maps
def compute_correlation_map(in_file, confounds, ctype):

    import numpy as np
    import nibabel as nb
    from scipy.stats import zscore

    # Load image
    img = nb.load(in_file)

    # zscore functional data
    data = zscore(img.get_fdata(), axis=-1)

    # zscore confound parameters
    par = zscore(np.loadtxt(
        confounds, skiprows='compcor' in confounds), axis=-1)

    # Compute correlation map per component
    corr_map=np.nan_to_num(np.dot(data, par))

    # Extract maximum correlation values per voxel
    corr_max = np.abs(corr_map).max(axis=-1)
    
    # Combine maximum correlation map with component correlation map
    corr_comb = np.rollaxis(np.vstack((
        corr_max[None,...], np.rollaxis(corr_map, -1, 0))), 0, 4)

    # Save and return output file
    img = nb.Nifti1Image(corr_comb, img.affine, img.header)
    out_file = in_file.replace('.nii', '_corr%s.nii' %ctype)
    img.to_filename(out_file)
    del img

    return out_file

comp_corr_mapA = Node(Function(input_names=['in_file', 'confounds', 'ctype'],
                               output_names=['out_file'],
                               function=compute_correlation_map),
                      name='comp_corr_mapA')
comp_corr_mapA.inputs.ctype = 'A'
comp_corr_mapT = comp_corr_mapA.clone('comp_corr_mapT')
comp_corr_mapT.inputs.ctype = 'T'
comp_corr_map_rp = comp_corr_mapA.clone('comp_corr_map_rp')
comp_corr_map_rp.inputs.ctype = 'RP'
comp_corr_map_friston = comp_corr_mapA.clone('comp_corr_map_friston')
comp_corr_map_friston.inputs.ctype = 'Friston'

In [ ]:
# Detects intensity and motion artifacts and labels them as outliers
art = Node(ArtifactDetect(norm_threshold=1,
                          zintensity_threshold=2.58, # corresponds to 99%
                          mask_type='file',
                          parameter_source='SPM',
                          use_differences=[True, False],
                          plot_type='svg'),
           name='art')

In [ ]:
# Computes Friston 24-parameter model (Friston et al., 1996)
def compute_friston24(in_file):
    
    import numpy as np
    
    # Load raw motion parameters
    mp_raw = np.loadtxt(in_file)
    
    # Get motion paremter one time point before
    mp_minus1 = np.vstack((mp_raw[1:], [0] * 6))
    
    # Combine the two
    mp_combine = np.hstack((mp_raw, mp_minus1))

    # Add the square of those parameters to allow correction of nonlinear effects
    mp_friston = np.hstack((mp_combine, mp_combine**2))

    # Save friston 24-parameter model in new txt file
    out_file = in_file.replace('.txt', '_friston24.txt')
    np.savetxt(out_file, mp_friston,
               fmt='%.8f', delimiter=' ', newline='\n')
    
    return out_file

friston24 = Node(Function(input_names=['in_file'],
                          output_names=['out_file'],
                          function=compute_friston24),
                 name='friston24')

### Create Confound Workflow

In [ ]:
# Create confound extraction workflow
confflow = Workflow(name='confflow')

In [ ]:
# Add nodes to workflow and connect them
confflow.connect([(acomp_masks, aCompCor, [('out_file', 'mask_files')]),
                  (tcomp_brainmask, tCompCor, [('out_file', 'mask_files')]),

                  # Compute confound correlation maps
                  (aCompCor, comp_corr_mapA, [('components_file', 'confounds')]),
                  (tCompCor, comp_corr_mapT, [('components_file', 'confounds')]),
                  (friston24, comp_corr_map_friston, [('out_file', 'confounds')]),
                  ])
confflow.add_nodes([art, comp_corr_map_rp])

## Create a subworkflow for the Visualization Workflow

### Implement Nodes

In [ ]:
# Create Title Text for visualization
def create_title(sub, sess, task, run):

    return 'Sub: %s - Task: %s - Sess: %s - Run: %s' % (
        sub, task, sess, run)

get_title = Node(Function(input_names=['sub', 'sess', 'task', 'run'],
                          output_names=['title'],
                          function=create_title),
                 name='get_title')

In [ ]:
# Visualize preprocessed functional mean on subject anatomy
def plot_mean(anat_file, mean_file, stype, title):

    from nibabel import load
    from nilearn.plotting import plot_stat_map

    # Create correct title
    title_text = title + ' - %s' % stype
    
    # Estimate threshold for nicer visualization
    thr = load(mean_file).get_fdata().max() * 0.05

    # Plot mean image overlayed on anatomy
    out_file = mean_file.replace('.nii.gz', '.svg')
    plot_stat_map(
        mean_file, title=title_text, bg_img=anat_file, colorbar=False,
        annotate=False, display_mode='ortho', threshold=thr, cut_coords=[0, 0, 0],
        draw_cross=False, symmetric_cbar= False, output_file=out_file)
    
    return out_file

vis_mean = Node(Function(input_names=['anat_file', 'mean_file',
                                      'stype', 'title'],
                         output_names=['out_file'],
                         function=plot_mean),
                name='vis_mean')
vis_mean.inputs.stype = 'FWHM'

vis_mean_bp = vis_mean.clone('vis_mean_bp')
vis_mean_bp.inputs.stype = 'BP'

In [ ]:
# Visualize estimated motion parameters
def plot_motion_parameters(mp_file, title):

    import seaborn as sns
    sns.set(context='notebook', style='darkgrid')
    import numpy as np
    import matplotlib.pyplot as plt

    # Load parameters and create figure
    mp_values = np.loadtxt(mp_file)
    fig, axes = plt.subplots(2, 1, figsize=(15, 5))
    axes[0].set_ylabel('rotation (radians)')
    axes[0].plot(mp_values[0:, 3:])
    axes[1].plot(mp_values[0:, :3])
    axes[1].set_xlabel('time (TR)')
    axes[1].set_ylabel('translation (mm)')
    axes[0].set_title(title + ' - Motion Parameters')
    plt.tight_layout()
    
    # Save output
    out_file = mp_file.replace('.txt','.svg')
    fig.savefig(out_file)
    
    return out_file

vis_mp = Node(Function(input_names=['mp_file', 'title'],
                       output_names=['out_file'],
                       function=plot_motion_parameters),
              name='vis_mp')

In [ ]:
# Visualize CompCor Masks
def plot_compcor_masks(anat_file, mask, mtype, title):

    import numpy as np
    import nibabel as nb
    from matplotlib.pyplot import cm
    from nilearn.plotting import plot_roi
    from nilearn.image import coord_transform

    # Estimate best cut coordinates
    img = nb.load(mask)
    idx = np.sort(img.get_fdata().nonzero()[1])
    if len(idx) != 0:
        vox_ids = np.linspace(idx[0], idx[-1], num=12, endpoint=True).astype('int')[2:-2]
        cut_ids = [int(coord_transform(0, r, 0, img.affine)[1]) for r in vox_ids]
    else:
        cut_ids = None

    # Visualize mask on anatomy
    out_file = mask.replace('.nii.gz', '.svg')
    plot_roi(mask, bg_img=anat_file, dim=1, colorbar=False, annotate=False,
             threshold=0.5, draw_cross=False, cmap=cm.bwr_r, display_mode='y',
             cut_coords=cut_ids, title=title + ' - Mask%s' % mtype,
             output_file=out_file)
    
    return out_file

vis_compmaskA = Node(Function(input_names=['anat_file', 'mask', 'mtype', 'title'],
                             output_names=['out_file'],
                             function=plot_compcor_masks),
                    name='vis_compmaskA')
vis_compmaskA.inputs.mtype = 'A'

vis_compmaskT = vis_compmaskA.clone('vis_compmaskT')
vis_compmaskT.inputs.mtype = 'T'

In [ ]:
# Visualize estimated compcor parameters
def plot_compcor_parameters(compcor_file, title, cType, ncomp):

    import seaborn as sns
    sns.set(context='notebook', style='darkgrid')
    import numpy as np
    import matplotlib.pyplot as plt

    # Load parameters and create figure
    comp = np.loadtxt(compcor_file, skiprows=1)
    fig = plt.figure(figsize=(15, 5))
    offset = [-r/3. for r in range(ncomp)]
    plt.plot(comp + offset)
    plt.ylabel('Components')
    plt.xlabel('Time [in volumes]')
    plt.title(title + 'CompCor%s' % (cType))
    plt.yticks(offset, ['comp%02d' % (c + 1) for c in range(ncomp)])
    plt.tight_layout()
    
    # Save output
    out_file = compcor_file.replace('.txt','.svg')
    fig.savefig(out_file)
    
    return out_file

vis_compA = Node(Function(input_names=['compcor_file', 'title',
                                       'cType', 'ncomp'],
                          output_names=['out_file'],
                          function=plot_compcor_parameters),
                 name='vis_compA')
vis_compA.inputs.cType = 'A'
vis_compA.inputs.ncomp = ncomp

vis_compT = vis_compA.clone('vis_compT')
vis_compT.inputs.cType = 'T'

In [ ]:
# Visualize Confound Correlation Maps
def plot_confounds(anat_file, corr_file, title, ctype, ncomp):

    import numpy as np
    from nibabel import load
    from nilearn.plotting import plot_stat_map
    from nilearn.image import coord_transform
    from matplotlib.pyplot import figure

    # Create correct title
    title_text = title + ' - %s' % ctype
    
    # Estimate threshold for nicer visualization
    img = load(corr_file)
    data = img.get_fdata()

    # Estimate best cut coordinates
    idx = np.sort(data.nonzero()[1])
    if len(idx) != 0:
        vox_ids = np.linspace(idx[0], idx[-1], num=12, endpoint=True).astype('int')[2:-2]
        cut_ids = [int(coord_transform(0, r, 0, img.affine)[1]) for r in vox_ids]
    else:
        cut_ids = None

    # Plot confound correlation map overlayed on anatomy
    if ctype[-1] in ['A', 'T']:
        nPlots = ncomp
    else:
        nPlots = 1

    fig = figure(figsize=(15, 2 * nPlots))
    for i in range(nPlots):
        ax = fig.add_subplot(nPlots, 1, i + 1)
        comp_img = img.slicer[..., i]
        thr = comp_img.get_fdata().max() * 0.25
        if i != 0:
            postfix = ' - Comp%02d' % i
        else:
            postfix = ''
        
        plot_stat_map(
            comp_img, title=title_text + postfix, bg_img=anat_file, annotate=False,
            colorbar=False, display_mode='y', threshold=thr, draw_cross=False,
            cut_coords=cut_ids, symmetric_cbar=False, axes=ax)

    # Save output
    out_file = corr_file.replace('.nii.gz', '.svg')
    fig.savefig(out_file, bbox_inches='tight', facecolor='black', frameon=True)

    return out_file

vis_corrA = Node(Function(input_names=['anat_file', 'corr_file',
                                       'title', 'ctype', 'ncomp'],
                         output_names=['out_file'],
                         function=plot_confounds),
                name='vis_corrA')
vis_corrA.inputs.ctype = 'CompCorA'
vis_corrA.inputs.ncomp = ncomp
vis_corrT = vis_corrA.clone('vis_corrT')
vis_corrT.inputs.ctype = 'CompCorT'
vis_corr_rp = vis_corrA.clone('vis_corr_rp')
vis_corr_rp.inputs.ctype = 'RP'
vis_corr_rp.inputs.ncomp = 6
vis_corr_friston = vis_corrA.clone('vis_corr_friston')
vis_corr_friston.inputs.ctype = 'Friston 24-parameter'
vis_corr_friston.inputs.ncomp = 24

### Create Visualization Workflow

In [ ]:
# Create visualization workflow
vizflow = Workflow(name='vizflow')

In [ ]:
# Add nodes to workflow and connect them
vizflow.connect([(get_title, vis_mp, [('title', 'title')]),
                 (get_title, vis_mean, [('title', 'title')]),
                 (get_title, vis_mean_bp, [('title', 'title')]),
                 (get_title, vis_compmaskA, [('title', 'title')]),
                 (get_title, vis_compmaskT, [('title', 'title')]),
                 (get_title, vis_compA, [('title', 'title')]),
                 (get_title, vis_compT, [('title', 'title')]),
                 (get_title, vis_corrA, [('title', 'title')]),
                 (get_title, vis_corrT, [('title', 'title')]),
                 (get_title, vis_corr_rp, [('title', 'title')]),
                 (get_title, vis_corr_friston, [('title', 'title')]),
                 ])

## Specify Input & Output Stream

In [ ]:
# Iterate over subject, session, task and run id
infosource = Node(IdentityInterface(fields=['subject_id', 'session_id',
                                            'task_id', 'run_id']),
                  name='infosource')
infosource.iterables = [('subject_id', subject_list),
                        ('session_id', session_list),
                        ('task_id', [task_id]),
                        ('run_id', run_list)]

In [ ]:
# Specify input file location
templates = {'anat':  opj('datasink', 'preproc_anat', 'sub-{subject_id}',
                          'ses-{session_id}_brain.nii.gz'),
             'wm':  opj('datasink', 'preproc_anat', 'sub-{subject_id}',
                        'ses-{session_id}_seg_wm.nii'),
             'csf':  opj('datasink', 'preproc_anat', 'sub-{subject_id}',
                         'ses-{session_id}_seg_csf.nii'),
             'func':  opj('/data', 'sub-{subject_id}', 'ses-{session_id}', 'func',
                          'sub-{subject_id}_ses-{session_id}_task-{task_id}_run-{run_id}_bold.nii.gz')}

sf = Node(SelectFiles(templates,
                      base_directory=exp_dir,
                      sort_filelist=True),
          name='selectfiles')

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [('_subject_id_', ''),
                 ('_session_id_', ''),
                 ('task_id_', ''),
                 ('run_id_', '')]
substitutions += [('_%s%s%s_%s/' % (run, sess, sub, task_id),
                   'sub-%s/task-%s_ses-%s_run-%s_' % (
                       sub, task_id, sess, run))
                  for sub in subject_list
                  for sess in session_list
                  for run in run_list]
substitutions += [
    ('sub-%s_ses-%s_task-%s_run-%s_bold' % (sub, sess, task_id, run), '')
    for sub in subject_list
    for sess in session_list
    for run in run_list]
substitutions += [('_ras', ''),
                  ('_TR', ''),
                  ('_maths', ''),
                  ('_filt', ''),
                  ('_flirt', ''),
                  ('_brain', ''),
                  ('_crop', ''),
                  ('ar_', ''),
                  ('_roi', ''),
                  ('rp_', 'rp'),
                  ('.r.', '.'),
                  ('mask_000', 'maskT'),
                  ('.r', ''),
                  ('art.r', 'art'),
                  ]
substitutions += [('_fwhm_%s/' % f, '') for f in fwhm]
datasink.inputs.substitutions = substitutions

## Implement Functional Preprocessing Workflow

In [ ]:
# Create functional preprocessing workflow
preproc_func = Workflow(name='preproc_func')
preproc_func.base_dir = opj(exp_dir, work_dir)

# Connect input nodes to each other
preproc_func.connect([(infosource, sf, [('subject_id', 'subject_id'),
                                        ('session_id', 'session_id'),
                                        ('task_id', 'task_id'),
                                        ('run_id', 'run_id')])])

In [ ]:
# Add input and output nodes and connect them to the main workflow
preproc_func.connect([(infosource, mainflow, [('task_id', 'getParam.task_id')]),
                      (sf, mainflow, [('func', 'reorient.in_file'),
                                      ('anat', 'coreg_pre.reference'),
                                      ('anat', 'coreg_bbr.reference'),
                                      ('wm', 'coreg_bbr.wm_seg'),
                                      ('anat', 'applycoreg.reference')]),
                      
                      (mainflow, datasink, [
                          ('realign.realignment_parameters', 'preproc_func.@realign'),
                          ('smooth_lowpass.out_file', 'preproc_func.@func'),
                          ('smooth_bandpass.out_file', 'preproc_func.@func_bp'),
                          ('meanimg.out_file', 'preproc_func.@mean'),
                          ('meanimg_bp.out_file', 'preproc_func.@mean_bp')]),
                      ])

In [ ]:
# Add input and output nodes and connect them to the confound workflow
preproc_func.connect([(sf, confflow, [('wm', 'acomp_masks.wm'),
                                      ('csf', 'acomp_masks.csf')]),
                      
                      (confflow, datasink, [
                          ('friston24.out_file', 'preproc_func.@friston24'),
                          ('art.outlier_files', 'preproc_func.@outlier_files'),
                          ('art.plot_files', 'viz_func.@plot_files'),
                          ('art.intensity_files', 'preproc_func.@intensity_files'),
                          ('art.norm_files', 'preproc_func.@norm_files'),
                          ('aCompCor.components_file', 'preproc_func.@compA'),
                          ('acomp_masks.out_file', 'preproc_func.@compA_mask'),
                          ('tCompCor.components_file', 'preproc_func.@compT'),
                          ('tCompCor.high_variance_masks', 'preproc_func.@compT_mask'),
                          ('comp_corr_mapA.out_file', 'preproc_func.@corr_compA'),
                          ('comp_corr_mapT.out_file', 'preproc_func.@corr_compT'),
                          ('comp_corr_map_rp.out_file', 'preproc_func.@corr_compRp'),
                          ('comp_corr_map_friston.out_file', 'preproc_func.@corr_compFriston')]),
                       ])

In [ ]:
# Add input and output nodes and connect them to the visualization workflow
preproc_func.connect([(infosource, vizflow, [('subject_id', 'get_title.sub'),
                                             ('session_id', 'get_title.sess'),
                                             ('task_id', 'get_title.task'),
                                             ('run_id', 'get_title.run')]),
                      (sf, vizflow, [('anat', 'vis_mean.anat_file'),
                                     ('anat', 'vis_mean_bp.anat_file'),
                                     ('anat', 'vis_compmaskA.anat_file'),
                                     ('anat', 'vis_compmaskT.anat_file'),
                                     ('anat', 'vis_corrA.anat_file'),
                                     ('anat', 'vis_corrT.anat_file'),
                                     ('anat', 'vis_corr_rp.anat_file'),
                                     ('anat', 'vis_corr_friston.anat_file')]),
                      
                      (vizflow, datasink, [
                          ('vis_mp.out_file', 'viz_func.@mp'),
                          ('vis_mean.out_file', 'viz_func.@vis_mean'),
                          ('vis_mean_bp.out_file', 'viz_func.@vis_mean_bp'),
                          ('vis_compmaskA.out_file', 'viz_func.@vis_compmaskA'),
                          ('vis_compmaskT.out_file', 'viz_func.@vis_compmaskT'),
                          ('vis_compA.out_file', 'viz_func.@vis_compA'),
                          ('vis_compT.out_file', 'viz_func.@vis_compT'),
                          ('vis_corrA.out_file', 'viz_func.@vis_corrA'),
                          ('vis_corrT.out_file', 'viz_func.@vis_corrT'),
                          ('vis_corr_rp.out_file', 'viz_func.@vis_corrRp'),
                          ('vis_corr_friston.out_file', 'viz_func.@vis_corrFriston')]),
                      ])

In [ ]:
# Connect main workflow with confound workflow
preproc_func.connect([(mainflow, confflow, [
                          ('getParam.TR', 'aCompCor.repetition_time'),
                          ('cropper.out_file', 'aCompCor.realigned_file'),
                          ('cropper.out_file', 'acomp_masks.in_file'),
                          ('getParam.TR', 'tCompCor.repetition_time'),
                          ('cropper.out_file', 'tCompCor.realigned_file'),
                          ('cropper.out_file', 'tcomp_brainmask.in_file'),
                          ('cropper.out_file', 'comp_corr_mapA.in_file'),
                          ('cropper.out_file', 'comp_corr_mapT.in_file'),
                          ('cropper.out_file', 'comp_corr_map_rp.in_file'),
                          ('cropper.out_file', 'comp_corr_map_friston.in_file'),
                          ('realign.realigned_files', 'art.realigned_files'),
                          ('realign.realignment_parameters', 'art.realignment_parameters'),
                          ('realign.realignment_parameters', 'comp_corr_map_rp.confounds'),
                          ('realign.realignment_parameters', 'friston24.in_file'),
                          ('bet_func.mask_file', 'art.mask_file')]),
                      ])

In [ ]:
# Connect main workflow with visualization workflow
preproc_func.connect([(mainflow, vizflow, [
                          ('realign.realignment_parameters', 'vis_mp.mp_file'),
                          ('meanimg.out_file', 'vis_mean.mean_file'),
                          ('meanimg_bp.out_file', 'vis_mean_bp.mean_file')]),
                      ])

In [ ]:
# Connect confound workflow with visualization workflow
preproc_func.connect([(confflow, vizflow, [
                          ('acomp_masks.out_file', 'vis_compmaskA.mask'),
                          ('tCompCor.high_variance_masks', 'vis_compmaskT.mask'),
                          ('aCompCor.components_file', 'vis_compA.compcor_file'),
                          ('tCompCor.components_file', 'vis_compT.compcor_file'),
                          ('comp_corr_mapA.out_file', 'vis_corrA.corr_file'),
                          ('comp_corr_mapT.out_file', 'vis_corrT.corr_file'),
                          ('comp_corr_map_rp.out_file', 'vis_corr_rp.corr_file'),
                          ('comp_corr_map_friston.out_file', 'vis_corr_friston.corr_file')]),
                      ])

## Visualize Workflow

In [ ]:
# Create preproc_func output graph
preproc_func.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename=opj(preproc_func.base_dir, 'preproc_func', 'graph.png'))

# Run Workflow

In [ ]:
# Run the workflow in parallel mode
preproc_func.run(plugin='MultiProc', plugin_args={'n_procs' : n_proc})

In [ ]:
# Save workflow graph visualizations in datasink
preproc_func.write_graph(graph2use='flat', format='svg', simple_form=True)
preproc_func.write_graph(graph2use='colored', format='svg', simple_form=True)

from shutil import copyfile
copyfile(opj(preproc_func.base_dir, 'preproc_func', 'graph.svg'),
         opj(exp_dir, out_dir, 'preproc_func', 'graph.svg'))
copyfile(opj(preproc_func.base_dir, 'preproc_func', 'graph_detailed.svg'),
         opj(exp_dir, out_dir, 'preproc_func', 'graph_detailed.svg'));

# Show Created Visualizations in Notebook

In [ ]:
from IPython.display import SVG

In [ ]:
# Visualize the mean image of the preprocessed functional data
for sub in subject_list:
    for sess in session_list:
        for run in run_list:

            for f in fwhm:
                display(SVG(opj(
                    exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                    'task-%s_ses-%s_run-%s_LP_%smm_mean.svg' % (task_id, sess, run, f))))

                display(SVG(opj(
                    exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                    'task-%s_ses-%s_run-%s_BP_%smm_mean.svg' % (task_id, sess, run, fwhm_bp))))

In [ ]:
# Visualize motion and intensity outliers
for sub in subject_list:
    for sess in session_list:
        for run in run_list:

            print('Outlier from: task-%s_ses-%s_run-%s' % (task_id, sess, run))
            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_plot.svg' % (task_id, sess, run))))


In [ ]:
# Visualize the masks used for CompCor approach
for sub in subject_list:
    for sess in session_list:
        for run in run_list:

            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_maskA.svg' % (task_id, sess, run))))
            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_maskT.svg' % (task_id, sess, run))))

In [ ]:
# Visualize the motion parameters and their correlation with the brain
for sub in subject_list:
    for sess in session_list:
        for run in run_list:

            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_rp.svg' % (task_id, sess, run))))
            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_corrRP.svg' % (task_id, sess, run))))
            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_corrFriston.svg' % (task_id, sess, run))))

In [ ]:
# Visualize the CompCor components and their brain correlates
for sub in subject_list:
    for sess in session_list:
        for run in run_list:

            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_compcorA.svg' % (task_id, sess, run))))
            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_corrA.svg' % (task_id, sess, run))))
            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_compcorT.svg' % (task_id, sess, run))))
            display(SVG(opj(
                exp_dir, out_dir, 'viz_func', 'sub-%s' % sub,
                'task-%s_ses-%s_run-%s_corrT.svg' % (task_id, sess, run))))

In [ ]:
display(SVG(opj(exp_dir, out_dir, 'preproc_func', 'graph.svg')))

In [ ]:
display(SVG(opj(exp_dir, out_dir, 'preproc_func', 'graph_detailed.svg')))